# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [341]:
# initial imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import panel as pn
import plotly.express as px
from pathlib import Path
from panel.interact import interact
from panel import widgets

from dotenv import load_dotenv


# Initialize the Panel Extensions (for Plotly)
pn.extension('plotly')

In [342]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")

# Set token using Plotly Express set function
px.set_mapbox_access_token(mapbox_token)


# Import Data

In [343]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")


file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)



- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [345]:
# Define Panel Visualization Functions
#Function to create housing unit plot

def create_housing_units_plot():
    """Housing Units Per Year."""
    housing_units = sfo_data.groupby('year').mean()
    housing_units.head(7)

    housing_units_plot = housing_units['housing_units'].hvplot.bar(
        ylim = (370000,387000), ylabel = "Housing Units",
        title = "Housing Units in San Francisco from 2010 to 2016")
    
#     plt.ylabel('Housing Units')
#     plt.title("Housing Units in San Francisco from 2010 to 2016", fontsize=14)
    
    return housing_units_plot  
        

  

In [347]:
#Function to create avg gross rent plot

def create_avg_gross_rent_plot():
    """Average Gross Rent in San Francisco Per Year."""
    
    housing_units = sfo_data.groupby('year').mean()
    avg_gross_rent = housing_units['gross_rent']

    avg_gross_rent_plot = avg_gross_rent.hvplot(
        title = "Average Gross Rent in San Francisco",
        ylabel = "Avg Gross Rent")
       
    return avg_gross_rent_plot


In [348]:
#Function to create avg sales price plot

def create_avg_sales_price_plot():
    """Average Sales Price Per Year."""
    housing_units = sfo_data.groupby('year').mean()
    avg_sales_price = housing_units['sale_price_sqr_foot']
  
    
    avg_sales_price_plot = avg_sales_price.hvplot(
        title = "Average Sales Price per Square Foot in San Francisco",
        ylabel = "Avg Sales Price")
    
    
    return avg_sales_price_plot



In [349]:
#Function to create avg price by neighborhood plot

def create_avg_price_by_neighborhood_plot():
    """Average Prices by Neighborhood."""
    
    neighborhood_df = sfo_data.groupby(['year', 'neighborhood']).mean()
    
    avg_price_by_neighborhood_plot = neighborhood_df.hvplot.line(
        
        y= 'sale_price_sqr_foot', 
        groupby='neighborhood', 
        ylabel = "Sales Price Per Square Foot"
    )

    return avg_price_by_neighborhood_plot



In [350]:
#Function to create top 10 most expensive neighborhood plot

def create_top10_plot():
    """Top 10 Most Expensive Neighborhoods."""

    top10_neighborhood_df = sfo_data.groupby(['neighborhood']).mean().sort_values(
        'sale_price_sqr_foot', ascending = False)
     
    top10_neighborhood_df.reset_index(inplace=True)
    
    top10_neighborhood_df = top10_neighborhood_df.head(10)    

    top10_plot = top10_neighborhood_df.hvplot.bar(
        
        x= 'neighborhood', 
        y = 'sale_price_sqr_foot', 
        figsize=(12,5), height = 500, rot = 90, 
        title = "Top 10 Most Expensive Neighborhoods",
        ylabel = "Avg Sales Price Per Square Foot"
    )
    
    return top10_plot


In [351]:
#Function to create parallel coordinates plot


top10_neighborhood_df = sfo_data.groupby(['neighborhood']).mean().sort_values('sale_price_sqr_foot', ascending = False)  
top10_neighborhood_df.reset_index(inplace=True)
top10_neighborhood_df.sort_values(['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent'])
top10_neighborhood_df = top10_neighborhood_df.head(10)

def create_parallel_coordinates():
    """Parallel Coordinates Plot."""
    
    parallel_coordinates = px.parallel_coordinates(
        top10_neighborhood_df,
        color = 'sale_price_sqr_foot')
    
    return parallel_coordinates



In [352]:
#Function to create parallel categories plot


top10_neighborhood_df = sfo_data.groupby(['neighborhood']).mean().sort_values('sale_price_sqr_foot', ascending = False)  
top10_neighborhood_df.reset_index(inplace=True)
top10_neighborhood_df.sort_values(['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent'])
top10_neighborhood_df = top10_neighborhood_df.head(10)   

def create_parallel_categories():
    """Parallel Categories Plot."""

    parallel_categories = px.parallel_categories(
        top10_neighborhood_df,
        dimensions=["neighborhood","sale_price_sqr_foot", "housing_units", "gross_rent"],
        color="sale_price_sqr_foot",
        color_continuous_scale=px.colors.sequential.Inferno,
        labels={
        "neighborhood": "Neighborhood",
        "sales_price_sqr_foot": "Sale price per square foot",
        "housing_units": "Housing Units",
        "gross_rent": "Gross Rent",
            },
        )
    
    return parallel_categories


In [354]:
#Function to create neighborhood map

def create_neighborhood_map():
    """Neighborhood Map"""
    neighborhood_df = sfo_data.groupby(['neighborhood']).mean()
    neighborhood_df = neighborhood_df.reset_index()
    neighborhood_df.rename(columns = {'neighborhood':'Neighborhood'}, inplace=True)
    neighborhood_merge_df= pd.merge(df_neighborhood_locations, neighborhood_df, on="Neighborhood", how='inner')
    
    
    return px.scatter_mapbox(
        neighborhood_merge_df,
        lat="Lat",
        lon="Lon",
        size="sale_price_sqr_foot",
        color="gross_rent",
        )
    


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [355]:
# Create panels to structure the layout of the dashboard

column1 = pn.Column(
    "## Yearly Housing Units and Avg Gross Rent Plots", create_housing_units_plot(), 
    create_avg_gross_rent_plot()
)

column2 = pn.Column(
    "## Avg Sales Price and Avg Price by Neighborhood", create_avg_sales_price_plot(), 
    create_avg_price_by_neighborhood_plot()
)

column3 = pn.Column(
    "## Top 10 and Neighborhood Map", create_top10_plot(), 
    create_neighborhood_map()
)

column4 = pn.Column(
    "## Parallel Coordinates and Parallel Categories",
    create_parallel_coordinates(),
    create_parallel_categories(),
)

# Create tabs

san_fran_dashboard = pn.Tabs(
    ("Yearly Unit and Rent Data", column1), ("Avg Sales Prices", column2), ("Top 10 Neighborhoods and Map", column3),
    ("Parallels", column4)
    
)



## Serve the Panel Dashboard

In [356]:
#Serve the dashboard

san_fran_dashboard.servable()



Tabs
    [0] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] HoloViews(Curve)
    [1] Column
        [0] Markdown(str)
        [1] HoloViews(Curve)
        [2] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
    [2] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] Plotly(Figure)
    [3] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)